# P_bar globe, based on P_bar deep dive

Need to try this coord interp again, but with the methods that seem promising from P_bar deep dive

## Housekeeping

In [2]:
import xarray as xr
import matplotlib.pyplot as plt
import matplotlib as mpl
from matplotlib.path import Path
import matplotlib.colors as colors
import pandas as pd
import numpy as np
from importlib import reload
import cartopy.crs as ccrs
import cmocean.cm as cmo
import gsw
import scipy.ndimage as filter
import scipy.interpolate as interpolate
from flox.xarray import xarray_reduce
from xgcm.autogenerate import generate_grid_ds
from xgcm import Grid
from tqdm.notebook import tqdm
from scipy.ndimage import uniform_filter1d

In [3]:
import os
os.chdir('/home/amf2288/argo-intern/funcs')
import filt_funcs as ff
import density_funcs as df

In [4]:
plt.rcParams['axes.titleweight'] = 'bold'
plt.rcParams['axes.titlesize']  = 18
plt.rcParams['axes.labelsize']  = 14
plt.rcParams['xtick.labelsize'] = 14
plt.rcParams['ytick.labelsize'] = 14
plt.rcParams['legend.fontsize'] = 14

## Load pre-made datasets

In [14]:
#individual profiles where sampling rate <=2.5m, pressure grid
ds_p = xr.open_dataset('/swot/SUM05/amf2288/res_2.5/ds_p.nc').rename({'PRES_INTERPOLATED':'PRESSURE','N_PROF':'PROFILE','LATITUDE':'LAT','LONGITUDE':'LON'})   
ds_p


<xarray.Dataset> Size: 95GB
Dimensions:          (PROFILE: 1488063, PRESSURE: 1001)
Coordinates:
  * PRESSURE         (PRESSURE) int64 8kB 0 2 4 6 8 ... 1992 1994 1996 1998 2000
    TIME             (PROFILE) datetime64[ns] 12MB ...
    LAT              (PROFILE) float64 12MB ...
    LON              (PROFILE) float64 12MB ...
  * PROFILE          (PROFILE) int64 12MB 2428701 1114461 ... 1502504 411205
Data variables:
    CT               (PROFILE, PRESSURE) float64 12GB ...
    CYCLE_NUMBER     (PROFILE, PRESSURE) float64 12GB ...
    PLATFORM_NUMBER  (PROFILE, PRESSURE) float64 12GB ...
    PRES             (PROFILE, PRESSURE) float32 6GB ...
    PSAL             (PROFILE, PRESSURE) float32 6GB ...
    SA               (PROFILE, PRESSURE) float64 12GB ...
    SIG0             (PROFILE, PRESSURE) float64 12GB ...
    TEMP             (PROFILE, PRESSURE) float32 6GB ...
    sample_rate      (PROFILE, PRESSURE) float32 6GB ...
    SPICE            (PROFILE, PRESSURE) float64 12GB ...
Attributes:
    DATA_ID:              ARGO
    DOI:                  http://doi.org/10.17882/42182
    Fetched_from:         https://data-argo.ifremer.fr
    Fetched_by:           amf2288
    Fetched_date:         2025/03/11
    Fetched_constraints:  [x=0.00/5.00; y=40.00/45.00; z=0.0/2002.0]
    Fetched_uri:          
    Valid uris:           ['https://data-argo.ifremer.fr/dac/coriolis/6903068...
    All uris:             ['https://data-argo.ifremer.fr/dac/coriolis/6903068...
    Processing_history:   [PRES,TEMP,PSAL] real-time and adjusted/delayed var...

## Sort by density and reindex

In [15]:
def sorting_index_from_sig(sig):
    """
    sig0: 1D numpy array for a single profile.
    Returns: 1D array of indices (same length) that sorts the valid region
             and leaves NaNs at the edges.
    """
    valid = ~np.isnan(sig)

    if not np.any(valid):
        return np.arange(len(sig))  # nothing to sort

    # valid region boundaries
    i0 = np.argmax(valid)
    i1 = len(sig) - np.argmax(valid[::-1])

    # sorting index for the core region
    core = sig[i0:i1]
    core_sort_idx = np.argsort(core)

    # build full index array
    full_idx = np.arange(len(sig))
    full_idx[i0:i1] = full_idx[i0:i1][core_sort_idx]

    return full_idx

In [16]:
def apply_sort_index(var, sort_idx, dim="PRESSURE"):
    return xr.apply_ufunc(
        lambda v, idx: v[idx],
        var,
        sort_idx,
        input_core_dims=[[dim], [dim]],
        output_core_dims=[[dim]],
        vectorize=True,
        dask="parallelized",
        output_dtypes=[var.dtype],
    )

In [17]:
sort_idx = xr.apply_ufunc(
    sorting_index_from_sig,
    ds_p["SIG0"],
    input_core_dims=[["PRESSURE"]],
    output_core_dims=[["PRESSURE"]],
    vectorize=True,
    dask="parallelized",
    output_dtypes=[int],
)

In [18]:
ds_p["SIG0_sort"] = apply_sort_index(ds_p["SIG0"], sort_idx)

In [19]:
#vars_to_sort = ["CT", "SA", "TEMP", "PSAL", "PRESSURE", "SPICE"]

#start with just CT for this notebook
vars_to_sort = ["CT"]

for v in vars_to_sort:
    ds_p[f"{v}_sort"] = apply_sort_index(ds_p[v], sort_idx)

In [20]:
#plots to compare CT to CT_sort on the globe

## Create ds_r

In [13]:
def interp_to_rho(rho_profile, var_profile, rho_grid, flag='group'):
    
    valid = ~np.isnan(rho_profile) & ~np.isnan(var_profile)
    rho_nonan = rho_profile[valid]
    var_nonan = var_profile[valid]

    if len(rho_nonan) < 3 or len(var_nonan) < 3:
        return np.full_like(rho_grid, np.nan)

    if flag == 'group':
        _, idx = np.unique(rho_nonan, return_index=True)
        rho_nonan = rho_nonan[idx]
        var_nonan = var_nonan[idx]

    fvar = interpolate.PchipInterpolator(rho_nonan, var_nonan, extrapolate=False)
    return fvar(rho_grid)

In [27]:
def get_ds_rho(ds_z, da_SIG, rho_grid, variables, dim1='PROFILE', dim2='PRESSURE'):

    xrs = []
    rho_size = rho_grid.size
    
    for var in variables:
        var_interp = xr.apply_ufunc(interp_to_rho, da_SIG, ds_z[var],
            input_core_dims=[[dim2], [dim2]],
            output_core_dims=[['rho_grid']],
            dask_gufunc_kwargs={'output_sizes': {'rho_grid': rho_size}},
            vectorize=True,
            dask='parallelized',
            kwargs={'rho_grid': rho_grid},
            output_dtypes=[ds_z[var].dtype])
        print(f'Completed comp for {var}')
        var_interp = xr.DataArray(
            data=var_interp.data,
            dims=[dim1, 'rho_grid'],
            coords={'rho_grid': rho_grid, dim1: ds_z[dim1]},
            name=var)
        xrs.append(var_interp)
        print(f'Completed {var}')
    
    ds_rho = xr.merge(xrs)
    print(f'Completed merge')
    ds_rho = ds_rho.assign_coords(LAT  =(dim1, ds_z.LAT.data))
    ds_rho = ds_rho.assign_coords(LON  =(dim1, ds_z.LON.data))
    ds_rho = ds_rho.assign_coords(TIME =(dim1, ds_z.TIME.data))
    
    return ds_rho

In [28]:
#again just doing CT for now, to see where we're at for method validation
variables = ['CT_sort','PRESSURE']
ds_r = get_ds_rho(ds_p, ds_p.SIG0_sort, np.arange(22,28,0.001), variables)

Completed comp for CT_sort
Completed CT_sort


/home/amf2288/miniconda3/envs/Argo_old_xgcm/lib/python3.11/site-packages/numpy/lib/function_base.py:2506: RuntimeWarning: invalid value encountered in cast
  output[index] = result


Completed comp for PRESSURE
Completed PRESSURE
Completed merge


## Create ds_pbar

In [35]:
def interp_to_pmean(var_profile, pmean_smooth, pmean_grid):

    try:
        valid = ~np.isnan(var_profile) & ~np.isnan(pmean_smooth)
        var_nonan = var_profile[valid]
        pmean_nonan = pmean_smooth[valid]

        fvar = interpolate.PchipInterpolator(pmean_nonan, var_nonan, extrapolate=False)
        return fvar(pmean_grid)

    except ValueError:
        return np.full_like(pmean_grid, np.nan)
        print(f"ValueError")

In [ ]:
def get_ds_pmean(ds_rho, pres_var, pmean_grid, variables, dim1='PROFILE', dim2='rho_grid'):
    
    xrs = []
    pmean_size = pmean_grid.size
    
    # 1. Mean pressure profile
    pres_profile = ds_rho[pres_var].mean(dim1)

    # 2. Sort the mean profile
    sort_idx_mean = sorting_index_from_sig(pres_profile.values)
    pres_profile_sorted = pres_profile.values[sort_idx_mean]

    # 3. Wrap back into DataArray
    pmean_smooth = xr.DataArray(
        pres_profile_sorted,
        dims=[dim2],
        coords={dim2: ds_rho[dim2]},
        name="pmean_smooth"
    )
    
    for var in variables:
        var_interp = xr.apply_ufunc(interp_to_pmean, ds_rho[var], pmean_smooth,
                                   input_core_dims=[[dim2], [dim2]],
                                   output_core_dims=[['Pmean_grid']],
                                   dask_gufunc_kwargs={'output_sizes': {'Pmean_grid':pmean_size}},
                                   vectorize=True,
                                   dask='parallaleized',
                                   kwargs={'pmean_grid':pmean_grid},
                                   output_dtypes=[ds_rho[var].dtype])
        print(f'Completed comp for {var}')
        var_interp = xr.DataArray(
            data=var_interp.data,
            dims=[dim1,'pmean_grid'],
            coords={'pmean_grid': pmean_grid, dim1: ds_rho[dim1]},
            name=var)
        xrs.append(var_interp)
        print(f'Completed {var}')
        
    ds_pmean = xr.merge(xrs)
    print(f'Completed merge')
    ds_pmean = ds_pmean.assign_coords(LAT = (dim1, ds_rho.LAT.data))
    ds_pmean = ds_pmean.assign_coords(LON = (dim1, ds_rho.LON.data))
    ds_pmean = ds_pmean.assign_coords(TIME = (dim1, ds_rho.TIME.data))
    
    return ds_pmean

In [37]:
ds_pbar = get_ds_pmean(ds_r, 'PRESSURE', np.linspace(0, 2000, 1001), variables)

Completed comp for CT_sort
Completed CT_sort


/home/amf2288/miniconda3/envs/Argo_old_xgcm/lib/python3.11/site-packages/numpy/lib/function_base.py:2506: RuntimeWarning: invalid value encountered in cast
  output[index] = result


Completed comp for PRESSURE
Completed PRESSURE
Completed merge


## Create gridded ds_p, ds_r, ds_pbar

In [41]:
def get_ds_gridded(ds, lon_bins, lat_bins, z_coord):
    # Step 1: Reduce using flox
    ds_binned = xarray_reduce(
        ds,
        'LON',
        'LAT',
        func='mean',
        expected_groups=(
            pd.IntervalIndex.from_breaks(lon_bins),
            pd.IntervalIndex.from_breaks(lat_bins)
        ),
        fill_value=np.nan,
        skipna=True
    )

    # Step 2: Rename dimensions and variables
    ds_binned = ds_binned.rename_dims({
        'LON_bins': 'LON',
        'LAT_bins': 'LAT',}).rename_vars({
        #'rho_grid': 'density'
        'LON_bins': 'LON',
        'LAT_bins': 'LAT',
        #'rho_grid': 'density'
    })

    # Step 3: Replace Interval coordinates with midpoints
    def interval_midpoints(intervals):
        return np.array([interval.mid for interval in intervals])

    ds_binned = ds_binned.assign_coords({
        'LON': ('LON', interval_midpoints(ds_binned['LON'].values)),
        'LAT': ('LAT', interval_midpoints(ds_binned['LAT'].values))
    })

    # Step 4: Generate grid and xgcm object
    ds_gridded = generate_grid_ds(ds_binned, {'X': 'LON', 'Y': 'LAT'})
    grid = Grid(ds_gridded, coords={
        'X': {'center': 'LON', 'left': 'lon_left'},
        'Y': {'center': 'LAT', 'left': 'lat_left'},
        'Z': {'center': z_coord}
    }, periodic=['X'])

    return ds_gridded, grid

In [39]:
lon_bins = np.arange(-180,181,3)
lat_bins = np.arange(-90,91,3)

In [40]:
ds_p_grid = get_ds_gridded(ds_p, lon_bins, lat_bins, 'PRESSURE')
print('Completed gridding ds_p')
ds_r_grid = get_ds_gridded(ds_r, lon_bins, lat_bins, 'rho_grid').rename({'rho_grid':'DENSITY'})
print('Completed gridding ds_r')
ds_pbar_grid = get_ds_gridded(ds_pbar, lon_bins, lat_bins, 'pmean_grid').rename({'pmean_grid':'PRESSURE_BAR'})
print('Completed gridding ds_pbar')

ValueError: cannot rename 'lon_bins' because it is not found in the dimensions of this dataset ('PRESSURE', 'LON_bins', 'LAT_bins')

## Plot results